<a href="https://colab.research.google.com/github/hosankang/AI_Class/blob/master/char_sentence_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function 
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [16]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [18]:
!ls /gdrive/My\ Drive/Colab\ Notebooks

 BatchNormalization.ipynb	      'plt Untitled1.ipynb'
 char_rnn.ipynb			       Regularizations.ipynb
 data				       resnet101_2.ipynb
 Fashion-MNIST_CNN.ipynb	       ResNet101_Fashion_MNIST.ipynb
'Fashion-MNIST_CNN.ipynb의 사본'       Untitled0.ipynb
'Fashion-MNIST_CNN.ipynb의 사본 (1)'   Untitled1.ipynb
 Fashion_MNIST.ipynb		       Untitled2.ipynb
 input.txt			       Untitled3.ipynb
 linux_kernel_code.txt		       Untitled4.ipynb


In [19]:
text = open('/gdrive/My Drive/Colab Notebooks/linux_kernel_code.txt', 'r').read().lower() #리눅스 c코드 
print('text length', len(text)) #코드 총 길이 

text length 20849


In [21]:
print(text[:1000]) #1000개만 확인 

#define pr_fmt(fmt) kbuild_modname ": " fmt
#include <linux/export.h>
#include <linux/errno.h>
#include <linux/ioport.h>
#include <linux/init.h>
#include <linux/slab.h>
#include <linux/spinlock.h>
#include <linux/fs.h>
#include <linux/proc_fs.h>
#include <linux/sched.h>
#include <linux/seq_file.h>
#include <linux/device.h>
#include <linux/pfn.h>
#include <linux/mm.h>
#include <linux/resource_ext.h>
#include <asm/io.h>

struct resource ioport_resource = {
	.name	= "pci io",
	.start	= 0,
	.end	= io_space_limit,
	.flags	= ioresource_io,
};
export_symbol(ioport_resource);

struct resource iomem_resource = {
	.name	= "pci mem",
	.start	= 0,
	.end	= -1,
	.flags	= ioresource_mem,
};

export_symbol(iomem_resource);

/* constraints to be met while allocating resources */
struct resource_constraint {
	resource_size_t min, max, align;
	resource_size_t (*alignf)(void *, const struct resource *,
			resource_size_t, resource_size_t);
	void *alignf_data;
};

static define_rwlock(resource_lock);

/*
 

In [22]:
chars = sorted(list(set(text))) #텍스트에 있는 고유 char들을 가져옴 
print(chars)
print('total chars: ', len(chars)) #고유 char의 갯수 

['\t', '\n', ' ', '!', '"', '#', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '4', '5', '6', '8', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']
total chars:  65


In [23]:
#2개의 사전 생성 
char_indices = dict((c,i)for i,c in enumerate(chars)) #char형 사전
indices_char = dict((i,c)for i,c in enumerate(chars)) #int형 사전
print(char_indices)
print('\n',indices_char)

{'\t': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, '#': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, '*': 11, '+': 12, ',': 13, '-': 14, '.': 15, '/': 16, '0': 17, '1': 18, '2': 19, '4': 20, '5': 21, '6': 22, '8': 23, ':': 24, ';': 25, '<': 26, '=': 27, '>': 28, '?': 29, '@': 30, '[': 31, '\\': 32, ']': 33, '_': 34, 'a': 35, 'b': 36, 'c': 37, 'd': 38, 'e': 39, 'f': 40, 'g': 41, 'h': 42, 'i': 43, 'j': 44, 'k': 45, 'l': 46, 'm': 47, 'n': 48, 'o': 49, 'p': 50, 'q': 51, 'r': 52, 's': 53, 't': 54, 'u': 55, 'v': 56, 'w': 57, 'x': 58, 'y': 59, 'z': 60, '{': 61, '|': 62, '}': 63, '~': 64}

 {0: '\t', 1: '\n', 2: ' ', 3: '!', 4: '"', 5: '#', 6: '%', 7: '&', 8: "'", 9: '(', 10: ')', 11: '*', 12: '+', 13: ',', 14: '-', 15: '.', 16: '/', 17: '0', 18: '1', 19: '2', 20: '4', 21: '5', 22: '6', 23: '8', 24: ':', 25: ';', 26: '<', 27: '=', 28: '>', 29: '?', 30: '@', 31: '[', 32: '\\', 33: ']', 34: '_', 35: 'a', 36: 'b', 37: 'c', 38: 'd', 39: 'e', 40: 'f', 41: 'g', 42: 'h', 43: 'i', 44: 'j', 45: 'k', 46: 'l', 47

In [74]:
#시퀀스
maxlen = 40 #시퀀스 40자 
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i+ maxlen])
    next_chars.append(text[i + maxlen])
print('sequences: ', len(sentences))

sequences:  6937


In [75]:
print(sentences[:3])
print(next_chars[:3])

['#define pr_fmt(fmt) kbuild_modname ": " ', 'fine pr_fmt(fmt) kbuild_modname ": " fmt', 'e pr_fmt(fmt) kbuild_modname ": " fmt\n#i']
['f', '\n', 'n']


In [0]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [77]:
#모델
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 128)               99328     
_________________________________________________________________
dense_6 (Dense)              (None, 65)                8385      
_________________________________________________________________
activation_6 (Activation)    (None, 65)                0         
Total params: 107,713
Trainable params: 107,713
Non-trainable params: 0
_________________________________________________________________


In [0]:
#모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
#학습마다 학습된 예측 코드 생성 
def on_epoch_end(epoch, logs):
    print()
    print('Epoch: %d '%epoch)
    start_index = random.randint(0, len(text) - maxlen -1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('diversity: ',diversity)
        
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)
        
        for i in range(300):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t,char in enumerate(sentence):
                x_pred[0, t, char_indices[char]]= 1
                
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            
            generated += next_char
            sentence = sentence[1:] + next_char
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [0]:
#체크포인트 콜백
from keras.callbacks import ModelCheckpoint

filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose =1,save_best_only=True, mode='min')

In [0]:
#Epoch마다 학습된 예상코드 불러오는 콜백
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience = 5, min_lr = 0.001)

callbacks = [print_callback, checkpoint, reduce_lr]

In [83]:
#학습 <배치 128, 에폭 40>
model.fit(x,y,batch_size= 128, epochs = 40, callbacks = callbacks)

Epoch 1/40
6937/6937 [==============================] - 7s 1ms/step - loss: 3.5814

Epoch: 0 
diversity:  0.2
Generating with seed: "id *arg, int (*func)(unsigned long, unsi"
id *arg, int (*func)(unsigned long, unsi      re  s           re          r    e        r e     cr     e       l          r       e   t        s   e     r  e        s        e  r   e s              r  e o r          e   e      r           or  r  r    i e  e  e  ee      e      r  s    o o  e       i oo    o     r     	r ee     e           
diversity:  0.5
Generating with seed: "id *arg, int (*func)(unsigned long, unsi"
id *arg, int (*func)(unsigned long, unsint o os r oee r=n   
 ieereoa_sn  c e e e c(erosrs  rruirt s m  t	 er* 
o iop t
us rosrtd  tsi
u_oiscfi_	sr  {  r  ce 	o rfercc  et or  r  a e>se  ir roego 
e_s ei no(r	e  elts  nsto  en  s_
e	 ems  ot co
  o }se r    .ereteei rp r	ife  e t i  snc
 c(er_st  lneni  tri russr eirereorc ueii ri e e_etd
diversity:  1.0
Generating with seed: "id *arg, int (*func)(un

In [0]:
#코드 생성 
def generate_text(length, diversity):
    start_index = random.randint(0, len(text) - maxlen -1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    for i in range(length):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t,char in enumerate(sentence):
            x_pred[0, t, char_indices[char]]= 1.
                
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
            
        generated += next_char
        sentence = sentence[1:] + next_char
    return generated

In [88]:
print('0.2')
print(generate_text(1000,0.2))
print('\n\n0.5:')
print(generate_text(1000,0.5))
print('\n\n1.0')
print(generate_text(1000,1.0))
print('\n\n1.2')
print(generate_text(1000,1.2))

0.2
is function is only for system ram, it desc, collres @flags = alloc.etd sestart at sorte tarestre  * @esource the resource start a tarsstre t alice the resource t peng f segne descresource tort, {
		struct resource * pont resource soret restrrestrust resource *p *

		 * resource_lock(&resource_lock);
	resurce_lock(&resource_lock);
	resurce_lick(&resource_lock);
	resurce_lock(&resource_lock);

	return pestarint = resource_lock);
	resource_lock);
	resurce_lick(&resturce_size_t resource *oor, tarestrust resource *p the resource start alinn foret resource *oot, start anstrestruc  resource * pont,
		         ounsist_desc_ins, vaid *ang, next ansize_t resource * @oll, stati  iol vem resource start ant resource *oor, testrestruct resource * @ollicestresturce struct resource *oot, testrestruct resource *p cond * source start anstrestrucs struct resource *p pend * source te size, resturce sorctiresorrest * @elogs *
 * @res: resource sizet  esturce struct resource *pont resource *oor, tew, s